Please run this section of the code to download the data on constellation borders. The csv file will contain the ra and dec that must be plotted (scatter) to obtain the borders of constellations

In [70]:
#First we import astroquery and remove the limit on rows.

from astroquery.vizier import Vizier
catalogue_list = Vizier.find_catalogs("Constellation")#This loads all the catalogs by the keyword "constellation"
Vizier.ROW_LIMIT = -1

catalog = Vizier.get_catalogs("VI/49")#This selects the 2nd of three tables that were found by the key-word search. This table contains relevant data
catalog[1].write("constellation_borders.csv",format="csv",overwrite="True")#creates a csv file

The output file generated by the above script will be stored in your working directory. to load tables directly from the Internet using URLs. For example, download tables from Vizier catalogues in CDS format ('ascii.cds'):

Run this section to download the ra,dec of messier objects

In [71]:
#PARSING THROUGH HYPERLEDA
#will add all mags into an array
mags = []

import requests
from bs4 import BeautifulSoup
for i in range(110):
    page = requests.get("http://leda.univ-lyon1.fr/ledacat.cgi?o=M{}".format(i+1))
    soup = BeautifulSoup(page.content,'html.parser')
    if len(soup.find_all('table',attrs={"class":"datatable"}))!=0:
        table_rel = soup.find_all('table',attrs={"class":"datatable"})[1]
        index = 0
        for rows in table_rel.find_all('tr'):
            if rows.find_all('a', attrs = {"title":"Total V-magnitude"}) != []:
                            mag_value = rows.find_all('td')[1].get_text().split()[0]
                            mags.append(mag_value)
                            index = index + 1
                            break
            if rows == table_rel.find_all('tr')[-1] and index == 0:
                mags.append('-')
                break
    else:
        mags.append('-')
        continue

#NEED TO ADD THESE VALUES TO THE MESSIER TABLE

In [72]:

#Import Simbad query
from astroquery.simbad import Simbad

#the next three lines are to simplify the output table
Simbad.reset_votable_fields()
Simbad.remove_votable_fields('coordinates')
Simbad.add_votable_fields('otype(S)', 'ra(d;A;ICRS;J2000;2000)', 'dec(d;D;ICRS;J2000;2000)','distance','flux(B)','flux(V)','flux_unit(V)')

result_table = Simbad.query_catalog("Messier")#This asks the SIMBAD database to list all objects from the messier catalog
result_table['V Mag (from HyperLeda/SED)'] = mags#adding v mags
result_table.write("messier_objects.csv",format="csv",overwrite="True")#creates a csv file

Simbad.reset_votable_fields()#renders the prev changes to simbad class temporary.

NGC Catalogue

In [69]:
#TAKES 3M 14S. QUITE SLOW. NEED A FASTER WAY OF DOING THINGS
#EXCLUDING STARS IT TAKES

from astroquery.simbad import Simbad
Simbad.TIMEOUT = 1000
Simbad.reset_votable_fields()
Simbad.remove_votable_fields('coordinates')
Simbad.add_votable_fields('otype(S)', 'ra(d;A;ICRS;J2000;2000)', 'dec(d;D;ICRS;J2000;2000)','distance','flux(B)','flux(V)')

result_table = Simbad.query_criteria('maintype != Star & Vmag <= 10 & cat =NGC')
result_table.write("NGC.csv",format="csv",overwrite="True")

Simbad.reset_votable_fields()

In [ ]:
import requests
from bs4 import BeautifulSoup
page = requests.get("")

Tycho Catalogue

In [ ]:
from astroquery.vizier import Vizier
catalogue_list = Vizier.find_catalogs("Tycho")#This loads all the catalogs by the keyword "constellation"
Vizier.ROW_LIMIT = -1
catalog_stars = Vizier.get_catalogs_async("I/239")
print(catalog_stars)

In [ ]:
from astropy.io import fits
from astropy.io import ascii


tab = fits.open("I_239_hip_main.dat.gz.fits")
data = tab[1].data
print(data)


parallax = data.field(11)
distance = [(1/float(x))*1000 if x!=0 else '-' for x in parallax]#distance in pc
ra  = data.field(8)
dec = data.field(9)
vmag = data.field(5)
slno = data.field(1)
specttype =  data.field(-2)
bvdiff = data.field(37)
hd = data.field(71)


#ascii.write([slno,hd,ra,dec,vmag,bvdiff,specttype,distance], 'hipparcos.csv',format="csv", names=['Sl.no.','HD Number','RA (deg)','Dec (deg)','V (mag)','B-V','Spectral Type','Distance (pc)'], overwrite=True)

Yale Bright Star

In [7]:
from astroquery.vizier import Vizier
catalog_strs = Vizier.get_catalogs("V/50")
Vizier.ROW_LIMIT = -1
catalog_strs[0].write("Yale_bright_star.csv",format="csv",overwrite="True")

Adding Star names to the catalogs using their HR names

In [67]:
import numpy as np

name_data = np.loadtxt("IAU-CSN.csv",delimiter=",", usecols=(1,3,4,9), dtype = np.str)
names = name_data[1:,0]
idc = name_data[1:,1]
cons = name_data[1:,2]
hdno = name_data[1:,3]

star_file = np.loadtxt("hipparcos.csv",delimiter=",", usecols=(1), dtype=np.str)
hd = star_file[1:]

common = np.where([x in hdno for x in hd])[0]
names_lst = np.array(['-' for x in hd])
names_lst[common] = names
        

['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-